In [1]:
import pandas as pd
import numpy as np

In [2]:
X_train=pd.read_csv('../input/month-dacon11/train_features.csv')
y_train=pd.read_csv('../input/month-dacon11/train_labels.csv')

samples=75
time_series=600
start_x=X_train.shape[0]-samples*time_series
X_train_new,X_test_new=X_train.iloc[:start_x],X_train.iloc[start_x:]

start_y=y_train.shape[0]-samples
y_train_new,y_test_new=y_train.iloc[:start_y],y_train.iloc[start_y:]
X_train_new.head(5)

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234


In [3]:
X_train_new=X_train_new.drop(['id','time'],axis=1)
X_test_new=X_test_new.drop(['id','time'], axis=1)

In [4]:
y_train_new.head(5)

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise


In [5]:
y_train_new=y_train_new.drop(['id','label_desc'],axis=1)
y_test_new=y_test_new.drop(['id','label_desc'],axis=1)

In [6]:
X_train_new=X_train_new.values
X_test_new=X_test_new.values
y_train_new=y_train_new.values
y_test_new=y_test_new.values
print('The size of X_train_new:', X_train_new.shape )
print('The size of X_test_new:', X_test_new.shape )

print('The size of y_train_new:', y_train_new.shape )
print('The size of y_test_new:', y_test_new.shape )

The size of X_train_new: (1830000, 6)
The size of X_test_new: (45000, 6)
The size of y_train_new: (3050, 1)
The size of y_test_new: (75, 1)


In [7]:
from keras.utils import to_categorical

y=np.concatenate([y_train_new,y_test_new])
y=to_categorical(y)

y_train_new = y[:-75]
y_test_new = y[-75:]
print('The shape of y_train_new:', y_train_new.shape)
print('The shape of y_test_new:', y_test_new.shape)

The shape of y_train_new: (3050, 61)
The shape of y_test_new: (75, 61)


In [8]:
nfeatures=X_train_new.shape[1]
ntimestamp=600
nsamples=3050
X_3D_train=X_train_new[:,0].reshape(nsamples,ntimestamp)
for i in range(nfeatures-1):
  i=i+1
  r=X_train_new[:,i].reshape(nsamples,ntimestamp)
  X_3D_train=np.dstack((X_3D_train,r))
print('The shape of X_train: ', X_3D_train.shape)

nfeatures=X_test_new.shape[1]
ntimestamp=600
nsamples=75
X_3D_test=X_test_new[:,0].reshape(nsamples,ntimestamp)
for i in range(nfeatures-1):
  i=i+1
  r=X_test_new[:,i].reshape(nsamples,ntimestamp)
  X_3D_test=np.dstack((X_3D_test,r))
print('The shape of X_test: ', X_3D_test.shape)

The shape of X_train:  (3050, 600, 6)
The shape of X_test:  (75, 600, 6)


In [9]:
nfeatures=X_3D_train.shape[2]
ntimestamp=X_3D_train.shape[1]
nsamples=X_3D_train.shape[0]

print('Number of features in Xtrain: ', nfeatures)
print('Number of timestamps in Xtrain: ', ntimestamp)
print('Number of samples in Xtrain: ', nsamples)

Number of features in Xtrain:  6
Number of timestamps in Xtrain:  600
Number of samples in Xtrain:  3050


In [10]:
for k in range(nfeatures):
    X_train_m = np.mean(X_3D_train[:,:,k])
    X_train_sd = np.std(X_3D_train[:,:,k])
    X_3D_train[:,:,k] = (X_3D_train[:,:,k]-X_train_m)/X_train_sd

for k in range(nfeatures):
    X_test_m = np.mean(X_3D_test[:,:,k])
    X_test_sd = np.std(X_3D_test[:,:,k])
    X_3D_test[:,:,k] = (X_3D_test[:,:,k]-X_test_m)/X_test_sd

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.layers import LSTM
from keras.layers import Activation
from keras import activations

from keras.layers.convolutional import Conv1D
from keras.optimizers import Adam
from keras.initializers import GlorotNormal

optimizer=Adam(learning_rate=0.0005)
initializer=GlorotNormal()
# https://keras.io/api/layers/initializers/

for k in range(1):
  verbose,epochs,batch_size=1,100,64
  model=Sequential()
  model.add(Conv1D(filters=16,kernel_size=3,
                   input_shape=(ntimestamp,nfeatures),
                   kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation(activations.relu))
  model.add(Conv1D(filters=32,kernel_size=3,kernel_initializer=initializer))
  model.add(BatchNormalization())
  model.add(Activation(activations.relu))
  model.add(LSTM(128))
  model.add(Dense(y_train_new.shape[1],activation='softmax'))
  model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
# fit network

In [28]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_24 (Conv1D)           (None, 598, 16)           304       
_________________________________________________________________
batch_normalization (BatchNo (None, 598, 16)           64        
_________________________________________________________________
activation (Activation)      (None, 598, 16)           0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 596, 32)           1568      
_________________________________________________________________
batch_normalization_1 (Batch (None, 596, 32)           128       
_________________________________________________________________
activation_1 (Activation)    (None, 596, 32)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)              

In [29]:
model.fit(X_3D_train,y_train_new,epochs=epochs,batch_size=batch_size,verbose=verbose)

Epoch 1/100
48/48 [==============================] - 6s 34ms/step - loss: 3.2580 - accuracy: 0.3978
Epoch 2/100
48/48 [==============================] - 2s 33ms/step - loss: 2.5486 - accuracy: 0.4900
Epoch 3/100
48/48 [==============================] - 2s 33ms/step - loss: 2.2833 - accuracy: 0.5038
Epoch 4/100
48/48 [==============================] - 2s 33ms/step - loss: 2.2161 - accuracy: 0.5142
Epoch 5/100
48/48 [==============================] - 2s 34ms/step - loss: 2.2698 - accuracy: 0.5133
Epoch 6/100
48/48 [==============================] - 2s 33ms/step - loss: 2.2497 - accuracy: 0.5132
Epoch 7/100
48/48 [==============================] - 2s 33ms/step - loss: 2.1461 - accuracy: 0.5275
Epoch 8/100
48/48 [==============================] - 2s 33ms/step - loss: 2.1145 - accuracy: 0.5259
Epoch 9/100
48/48 [==============================] - 2s 33ms/step - loss: 1.9475 - accuracy: 0.5445
Epoch 10/100
48/48 [==============================] - 2s 33ms/step - loss: 1.9152 - accuracy: 0.5611

48/48 [==============================] - 2s 33ms/step - loss: 0.7863 - accuracy: 0.7772
Epoch 83/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7658 - accuracy: 0.7805
Epoch 84/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7814 - accuracy: 0.7812
Epoch 85/100
48/48 [==============================] - 2s 34ms/step - loss: 0.8270 - accuracy: 0.7655
Epoch 86/100
48/48 [==============================] - 2s 34ms/step - loss: 0.8305 - accuracy: 0.7550
Epoch 87/100
48/48 [==============================] - 2s 34ms/step - loss: 0.7906 - accuracy: 0.7656
Epoch 88/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7659 - accuracy: 0.7733
Epoch 89/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7958 - accuracy: 0.7631
Epoch 90/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7314 - accuracy: 0.7854
Epoch 91/100
48/48 [==============================] - 2s 33ms/step - loss: 0.7621 - accuracy: 0.7824
Epo

In [51]:
# evaluate
np.mean(np.argmax(yhat,axis=1)==np.argmax(y_test_new,axis=1))

0.6

In [53]:
test=pd.read_csv('../input/month-dacon11/test_features.csv')

X_test_3D=test.drop(['id','time'],axis=1).values.reshape(-1,600,6)

In [76]:
pred=model.predict(X_test_3D)

In [77]:
submit=pd.read_csv('../input/month-dacon11/sample_submission.csv')
submit.iloc[:,1:]=pred
submit.head()

,id,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,3125,0.000880,0.000142,0.000006,0.000095,0.001231,0.004425,0.000014,0.000148,1.905429e-06,...,0.000026,8.060777e-08,0.000089,1.872273e-06,0.000145,0.000066,0.000006,9.374644e-07,0.009808,4.455566e-07
1,3126,0.000468,0.000377,0.000004,0.000032,0.000900,0.000323,0.000156,0.000499,6.487138e-07,...,0.000008,4.524130e-08,0.000006,6.125589e-07,0.000047,0.000007,0.000111,8.640509e-07,0.154934,3.810322e-06
2,3127,0.000879,0.000158,0.000001,0.000008,0.003097,0.001298,0.000006,0.001082,5.130681e-07,...,0.000009,1.585317e-07,0.000012,1.661962e-07,0.000008,0.000034,0.000019,4.832122e-07,0.008822,9.634643e-07
3,3128,0.026090,0.001311,0.000030,0.000405,0.004536,0.004897,0.000108,0.012751,3.072149e-06,...,0.000621,7.524662e-07,0.000370,4.098767e-06,0.000255,0.000027,0.000534,7.234331e-06,0.047945,9.016141e-06
4,3129,0.008258,0.000185,0.000003,0.000031,0.004468,0.003955,0.000025,0.010643,1.062241e-06,...,0.000396,2.534847e-07,0.000094,7.082789e-07,0.000029,0.000033,0.000042,9.665177e-07,0.003426,9.975045e-07


In [78]:
submit.to_csv('cnn_lstm_1.csv',index=False)